In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Processing results
import json

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
from sklearn.linear_model import LinearRegression

# Processing
from sklearn.preprocessing import StandardScaler

# Data imputation
from sklearn.impute import SimpleImputer

# Pipeline
from sklearn.pipeline import Pipeline

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score


# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

<IPython.core.display.Javascript object>

# Functions and definitions

In [3]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [4]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_ii",
    "Features": "Chemical + Mineralogical + CS3",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "Linear Regression",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [5]:
df = pd.read_csv("../../../../../../data/processed/partner_ii/cement-shipping.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [6]:
df_copy = df.drop(
    [
        # Properties
        
        
        "Initial setting time",
        "Blaine",
        "Sieve 32 um",
        "Sieve 45 um",
        "CS1",
        "CS7",
        "Cement_Type",
    ],
    axis=1,
).copy()

<IPython.core.display.Javascript object>

# 1. Linear Regression

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [7]:
y = df_copy.pop("CS28").values
x = df_copy.drop(["Date"], axis=1)
dates = df["Date"].copy()

<IPython.core.display.Javascript object>

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [8]:
repeats = 3
n_splits = 5
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Repeated KFold"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Repeated Cross Validation:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.714 (0.022)
MAE: -1.359 (0.016)
MAPE: -0.032 (0.000)
R2: 0.764 (0.005)


******
[TEST]
******
RMSE: -1.755 (0.092)
MAE: -1.389 (0.067)
MAPE: -0.033 (0.002)
R2: 0.749 (0.022)




<IPython.core.display.Javascript object>

In [9]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model Cross Validation  \
                                                                        
0  Chemical + Mineralogical + CS3  Linear Regression   Repeated KFold   

  RMSE Test           MAE Test           MAPE Test             R2 Test  \
       mean       std     mean       std      mean       std      mean   
0  1.754665  0.092025  1.38896  0.067464  0.032702  0.001645  0.749482   

             
        std  
0  0.021987

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [10]:
n_splits = 5
train_size = 0.8

pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
results_dict_copy[
    "Cross Validation Params"
] = '{"N_Splits": 5, "Repeats": 1, "train_size": 0.8}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.453 (0.156)
MAE: -1.142 (0.109)
MAPE: -0.027 (0.002)
R2: 0.773 (0.068)


******
[TEST]
******
RMSE: -2.032 (0.292)
MAE: -1.671 (0.247)
MAPE: -0.039 (0.005)
R2: 0.562 (0.139)




<IPython.core.display.Javascript object>

In [11]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                       
0  Chemical + Mineralogical + CS3  Linear Regression   
1  Chemical + Mineralogical + CS3  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  2.031680  0.292372  1.670933  0.246970   
1              Repeated KFold  1.754665  0.092025  1.388960  0.067464   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.038550  0.005317  0.561640  0.139376  
1  0.032702  0.001645  0.749482  0.021987

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [12]:
n_splits = 5
gap = 0
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = TimeSeriesSplit(gap=gap, max_train_size=None, n_splits=n_splits, test_size=None)

scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Time Series Split"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 1, "Gap": 0}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.475 (0.118)
MAE: -1.169 (0.087)
MAPE: -0.027 (0.002)
R2: 0.804 (0.038)


******
[TEST]
******
RMSE: -2.030 (0.426)
MAE: -1.646 (0.394)
MAPE: -0.039 (0.012)
R2: 0.510 (0.283)




<IPython.core.display.Javascript object>

In [13]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                       
0  Chemical + Mineralogical + CS3  Linear Regression   
1  Chemical + Mineralogical + CS3  Linear Regression   
2  Chemical + Mineralogical + CS3  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  2.031680  0.292372  1.670933  0.246970   
1              Repeated KFold  1.754665  0.092025  1.388960  0.067464   
2           Time Series Split  2.030383  0.426008  1.646351  0.394417   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.038550  0.005317  0.561640  0.139376  
1  0.032702  0.001645  0.749482  0.021987  
2  0.039343  0.011996  0.510287  0.282596

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [14]:
test_size = 0.2

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=SEED, shuffle=False
)
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)

pipeline.fit(x_train, y_train)

y_train_pred = pipeline.predict(x_train)
y_test_pred = pipeline.predict(x_test)

scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Out of time Split"
results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(
    results_dict_copy, {key: [value] for key, value in scores.items()}
)
results_to_save.append(df_results)

******
[TRAIN]
******
RMSE: 1.580 (0.000)
MAE: 1.249 (0.000)
MAPE: 0.029 (0.000)
R2: 0.765 (0.000)


******
[TEST]
******
RMSE: 2.766 (0.000)
MAE: 2.277 (0.000)
MAPE: 0.059 (0.000)
R2: 0.013 (0.000)




<IPython.core.display.Javascript object>

In [15]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                       
0  Chemical + Mineralogical + CS3  Linear Regression   
1  Chemical + Mineralogical + CS3  Linear Regression   
2  Chemical + Mineralogical + CS3  Linear Regression   
3  Chemical + Mineralogical + CS3  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  2.031680  0.292372  1.670933  0.246970   
1           Out of time Split  2.765725  0.000000  2.277434  0.000000   
2              Repeated KFold  1.754665  0.092025  1.388960  0.067464   
3           Time Series Split  2.030383  0.426008  1.646351  0.394417   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.038550  0.005317  0.561640  0.139376  
1  0.059109  0.000000  0.012632  0.000000  
2  0.032702  0.001645  0.749482  0.021987  
3  0.039343  0.011996  0.510287  0.282596

<IPython.core.display.Javascript object>

In [16]:
pipeline.named_steps["estimator"].coef_

array([ 0.05073059,  0.03155273,  0.08640629,  0.06451272, -0.07605561,
       -0.26054463, -0.39865421, -0.24652329,  0.31934389,  0.54671688,
       -0.08559538, -0.08380257,  0.02526351, -0.11308192,  2.98513468])

<IPython.core.display.Javascript object>

In [17]:
pipeline.named_steps["estimator"].intercept_

43.77724465558195

<IPython.core.display.Javascript object>

In [18]:
coeff = pipeline.named_steps["estimator"].coef_
np.array(x.columns)[coeff == 0]

array([], dtype=object)

<IPython.core.display.Javascript object>

In [19]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns

Index(['CaO', 'SiO2', 'Al2O3', 'Fe2O3', 'MgO', 'SO3', 'K2O', 'Na2O',
       'Total C3S', 'Total C2S', 'C3A', 'C4AF', 'Free CaO', 'Loss on Ignition',
       'CS3'],
      dtype='object')

<IPython.core.display.Javascript object>

In [20]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns.shape

(15,)

<IPython.core.display.Javascript object>

In [21]:
x.columns.shape

(15,)

<IPython.core.display.Javascript object>

In [22]:
coeffs = pd.DataFrame(
    {col: [c] for col, c in zip(x.columns, coeff)}, index=["Coefficients"]
)

<IPython.core.display.Javascript object>

In [23]:
coeffs.T["Coefficients"].sort_values(ascending=False).to_frame(
    name="Coefficients"
).style.background_gradient(axis=None, vmin=1, vmax=5, cmap="Greens")

,Coefficients
CS3,2.985135
Total C2S,0.546717
Total C3S,0.319344
Al2O3,0.086406
Fe2O3,0.064513
CaO,0.050731
SiO2,0.031553
Free CaO,0.025264
MgO,-0.076056
C4AF,-0.083803


<IPython.core.display.Javascript object>

In [24]:
pd.concat(results_to_save).groupby(
    ["Features", "Model", "Cross Validation", "Cross Validation Params"]
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                       
0  Chemical + Mineralogical + CS3  Linear Regression   
1  Chemical + Mineralogical + CS3  Linear Regression   
2  Chemical + Mineralogical + CS3  Linear Regression   
3  Chemical + Mineralogical + CS3  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Test            \
                                                         mean       std   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}  2.031680  0.292372   
1                                {"Test Size": 0.2}  2.765725  0.000000   
2                     {"N_Splits": 5, "Repeats": 3}  1.754665  0.092025   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}  2.030383  0.426008   

   MAE Test           MAPE Test             R2 Test            
       mean       std      mean       std      mean       std  
0  1.670933  0.246970  0.038550  0.005317  0.561640  0.139376  
1  2.277434  0.000000  0.059109  0.000000  0.012632  0.000000  
2  1.388960  0.067464  0.032702  0.001645  0.749482  0.021987  
3  1.646351  0.394417  0.039343  0.011996  0.510287  0.282596

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [25]:
path = "../../../../../../reports/results/local_models/partner_ii/all_cements/full/"
filename = "linear_regression_results_full_12.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [26]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../../reports/results/local_models/partner_ii/all_cements/grouped/"
filename = "linear_regression_results_grouped_12.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>